In [1]:
!apt-get update && apt-get install -y build-essential libpython3-dev

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,618 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,323 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64

In [2]:
! git clone https://github.com/state-spaces/mamba

Cloning into 'mamba'...
remote: Enumerating objects: 648, done.
remote: Counting objects: 100% (300/300), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 648 (delta 228), reused 206 (delta 194), pack-reused 348 (from 1)
Receiving objects: 100% (648/648), 1.55 MiB | 6.96 MiB/s, done.
Resolving deltas: 100% (339/339), done.


In [3]:
!pip install ./mamba

Processing ./mamba
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl.metadata (5.3 kB)
  Using cached triton-3.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
Using cached triton-3.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (209.5 MB)
  Created wheel for mamba_ssm: filename=mamba_ssm-2.2.2-cp310-cp310-linux_x86_64.whl size=322978734 sha256=0e2e4fc3d41ca1ad6ae1812f28d96412a4a31682b9d16099448f0b000d626ff5
  Stored in directory: /tmp/pip-ephem-wheel-cache-2e3_s3zj/wheels/fb/bd/db/47d39b5aa5f1c5ccf671710c66e7a084dd82fb317f85f52625
Successfully built mamba_ssm


In [5]:
import argparse
import os.path as osp
from typing import Any, Dict, Optional
import math
import torch as th

import torch
from torch.nn import (
    BatchNorm1d,
    Embedding,
    Linear,
    ModuleList,
    ReLU,
    Sequential,
)
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torch_geometric.transforms as T
from torch_geometric.datasets import ZINC
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINEConv, global_add_pool
import inspect
from typing import Any, Dict, Optional

import torch.nn.functional as F
from torch import Tensor
from torch.nn import Dropout, Linear, Sequential

from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.inits import reset
from torch_geometric.nn.resolver import (
    activation_resolver,
    normalization_resolver,
)
from torch_geometric.typing import Adj
from torch_geometric.utils import to_dense_batch

from mamba_ssm import Mamba
from torch_geometric.utils import degree, sort_edge_index
import torch_sparse
#import torch_geometric

In [ ]:
!pip uninstall torch-sparse torch-geometric torch-scatter torch-cluster

Found existing installation: torch_sparse 0.6.18
Uninstalling torch_sparse-0.6.18:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/torch_sparse-0.6.18.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch_sparse/*
Proceed (Y/n)? Y
  Successfully uninstalled torch_sparse-0.6.18
Found existing installation: torch_geometric 2.3.1
Uninstalling torch_geometric-2.3.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/torch_geometric-2.3.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch_geometric/*
Proceed (Y/n)? Y
  Successfully uninstalled torch_geometric-2.3.1
Found existing installation: torch_scatter 2.1.2
Uninstalling torch_scatter-2.1.2:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/torch_scatter-2.1.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch_scatter/*
Proceed (Y/n)? Y
  Successfully uninstalled torch_scatter-2.1.2


In [6]:
import torch
print(torch.__version__)

2.5.1+cu121


In [4]:
!pip install torch-sparse torch-scatter torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-2.4.1+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.4.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 119.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 986.2/986.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.5 MB/s eta 0:00:00


In [7]:
path, subset = '/scratch/ssd004/scratch/tsepaole/ZINC_full/', False
path, subset = '', True

transform = T.AddRandomWalkPE(walk_length=20, attr_name='pe')
train_dataset = ZINC(path, subset=subset, split='train', pre_transform=transform)
val_dataset = ZINC(path, subset=subset, split='val', pre_transform=transform)
test_dataset = ZINC(path, subset=subset, split='test', pre_transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

Extracting ./molecules.zip
Processing...
Processing test dataset: 100%|██████████| 1000/1000 [00:00<00:00, 1459.78it/s]
Done!


In [8]:
def timestep_embedding(timesteps, dim, max_period=10000):
      """
    Create sinusoidal timestep embeddings.

    :param timesteps: a 1-D Tensor of N indices, one per batch element.
                      These may be fractional.
    :param dim: the dimension of the output.
    :param max_period: controls the minimum frequency of the embeddings.
    :return: an [N x dim] Tensor of positional embeddings.
    """
      half = dim // 2
      freqs = th.exp(
        -math.log(max_period) * th.arange(start=0, end=half, dtype=th.float32) / half
      ).to(device=timesteps.device)
      args = timesteps[:, None].float() * freqs[None]
      embedding = th.cat([th.cos(args), th.sin(args)], dim=-1)
      if dim % 2:
        embedding = th.cat([embedding, th.zeros_like(embedding[:, :1])], dim=-1)
      return embedding

In [9]:
class GPSConv(torch.nn.Module):

    def __init__(
        self,
        channels: int,
        conv: Optional[MessagePassing],
        heads: int = 1,
        dropout: float = 0.0,
        attn_dropout: float = 0.0,
        act: str = 'relu',
        att_type: str = 'transformer',
        order_by_degree: bool = False,
        shuffle_ind: int = 0,
        d_state: int = 16,
        d_conv: int = 4,
        act_kwargs: Optional[Dict[str, Any]] = None,
        norm: Optional[str] = 'batch_norm',
        norm_kwargs: Optional[Dict[str, Any]] = None,
    ):
        super().__init__()

        self.channels = channels
        self.conv = conv
        self.heads = heads
        self.dropout = dropout
        self.att_type = att_type
        self.shuffle_ind = shuffle_ind
        self.order_by_degree = order_by_degree

        assert (self.order_by_degree==True and self.shuffle_ind==0) or (self.order_by_degree==False), f'order_by_degree={self.order_by_degree} and shuffle_ind={self.shuffle_ind}'
        # order_by_degree flag is set to True, this section reorders the nodes based on their degrees
        if self.att_type == 'transformer':
            self.attn = torch.nn.MultiheadAttention(
                channels,
                heads,
                dropout=attn_dropout,
                batch_first=True,
            )
        if self.att_type == 'mamba':
            self.self_attn = Mamba(
                d_model=channels,
                d_state=d_state,
                d_conv=d_conv,
                expand=1
            )

        self.mlp = Sequential(
            Linear(channels, channels * 2),
            activation_resolver(act, **(act_kwargs or {})),
            Dropout(dropout),
            Linear(channels * 2, channels),
            Dropout(dropout),
        )

        norm_kwargs = norm_kwargs or {}
        self.norm1 = normalization_resolver(norm, channels, **norm_kwargs)
        self.norm2 = normalization_resolver(norm, channels, **norm_kwargs)
        self.norm3 = normalization_resolver(norm, channels, **norm_kwargs)

        self.norm_with_batch = False
        if self.norm1 is not None:
            signature = inspect.signature(self.norm1.forward)
            self.norm_with_batch = 'batch' in signature.parameters

    def reset_parameters(self):
        r"""Resets all learnable parameters of the module."""
        if self.conv is not None:
            self.conv.reset_parameters()
        self.attn._reset_parameters()
        reset(self.mlp)
        if self.norm1 is not None:
            self.norm1.reset_parameters()
        if self.norm2 is not None:
            self.norm2.reset_parameters()
        if self.norm3 is not None:
            self.norm3.reset_parameters()

    def forward(
        self,
        x: Tensor,
        edge_index: Adj,
        batch: Optional[torch.Tensor] = None,
        **kwargs,
    ) -> Tensor:
        r"""Runs the forward pass of the module."""
        hs = []
        if self.conv is not None:  # Local MPNN.
            h = self.conv(x, edge_index, **kwargs)
            h = F.dropout(h, p=self.dropout, training=self.training)
            h = h + x
            if self.norm1 is not None:
                if self.norm_with_batch:
                    h = self.norm1(h, batch=batch)
                else:
                    h = self.norm1(h)
            hs.append(h)

        ### Global attention transformer-style model.
        if self.att_type == 'transformer':
            h, mask = to_dense_batch(x, batch)
            h, _ = self.attn(h, h, h, key_padding_mask=~mask, need_weights=False)
            h = h[mask]

        if self.att_type == 'mamba':

            if self.order_by_degree:
              # degree() function: edge_index[0]= source nodes of the edges, x.shape[0]= number of nodes)
                deg = degree(edge_index[0], x.shape[0]).to(torch.long)
                order_tensor = torch.stack([batch, deg], 1).T
                # creates a tensor by stacking the 'batch' tensor(batch_index of each node) and the degree of each node. making it easier to reorder nodes
                _, x = sort_edge_index(order_tensor, edge_attr=x)
                # reorders the feature tensor x based on the node degree and batch information, reordering x as well.

            if self.shuffle_ind == 0:
              # no shuffling occurs, and dense batch repr. is created from x
                h, mask = to_dense_batch(x, batch)
                h = self.self_attn(h)[mask]
                # the result is masked to remove invalid entries (padded entries)
                #
            else:
                mamba_arr = []
                # list to store the results of multiple attention passes over shuffled node features
                for _ in range(self.shuffle_ind):
                  # loops over the number of shuffles(shuffle_ind)
                    h_ind_perm = permute_within_batch(x, batch)
                    # Similar to before, converts the shuffled x into a dense representation for each bat
                    h_i, mask = to_dense_batch(x[h_ind_perm], batch)
                    # Similar to before, converts the shuffled x into a dense representation for each batch.
                    h_i = self.self_attn(h_i)[mask][h_ind_perm]
                    # The shuffled dense batch is passed through the self-attention mechanism, and the result is reordered back to the original node order (h_ind_perm).
                    mamba_arr.append(h_i)
                h = sum(mamba_arr) / self.shuffle_ind

                # Averages the results from all shuffled versions to produce the final node representations.
        ###

        h = F.dropout(h, p=self.dropout, training=self.training)
        h = h + x  # Residual connection.
        if self.norm2 is not None:
            if self.norm_with_batch:
                h = self.norm2(h, batch=batch)
            else:
                h = self.norm2(h)
        hs.append(h)

        out = sum(hs)  # Combine local and global outputs.

        out = out + self.mlp(out)
        if self.norm3 is not None:
            if self.norm_with_batch:
                out = self.norm3(out, batch=batch)
            else:
                out = self.norm3(out)

        return out

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.channels}, '
                f'conv={self.conv}, heads={self.heads})')

In [84]:
class GraphModel(torch.nn.Module):
    def __init__(self, channels: int, pe_dim: int, context_dim:int, num_layers: int, model_type: str, shuffle_ind: int, d_state: int, d_conv: int, order_by_degree: False, time_embed_dim: int):
        super().__init__()

        self.node_emb = Embedding(28, channels - pe_dim - time_embed_dim - context_dim)
        self.context_emb = Linear(1, context_dim)
        self.pe_lin = Linear(20, pe_dim)
        self.time_embed_dim = time_embed_dim
        self.context_dim = context_dim
        self.pe_norm = BatchNorm1d(20)
        self.edge_emb = Embedding(4, channels)
        self.model_type = model_type
        self.shuffle_ind = shuffle_ind
        self.order_by_degree = order_by_degree

        self.convs = ModuleList()


        for _ in range(num_layers):
            nn = Sequential(
                Linear(channels, channels),
                ReLU(),
                Linear(channels, channels),
            )
            if self.model_type == 'gine':
                conv = GINEConv(nn)

            if self.model_type == 'mamba':
                conv = GPSConv(channels, GINEConv(nn), heads=4, attn_dropout=0.5,
                               att_type='mamba',
                               shuffle_ind=self.shuffle_ind,
                               order_by_degree=self.order_by_degree,
                               d_state=d_state, d_conv=d_conv)

            if self.model_type == 'transformer':
                conv = GPSConv(channels, GINEConv(nn), heads=4, attn_dropout=0.5, att_type='transformer')

            # conv = GINEConv(nn)
            self.convs.append(conv)

            self.mlp = Sequential(
            Linear(channels, channels // 2),
            ReLU(),
            Linear(channels // 2, channels // 4),
            ReLU(),
            Linear(channels // 4, 1),
        )

    def forward(self, x, pe, edge_index, t, context, edge_attr, batch):

        x_pe = self.pe_norm(pe)
        t = t.view(-1, 1)
        t = t[batch_]
        time_emb = timestep_embedding(t, self.time_embed_dim)
        context = context.view(-1, context.size(-1))
        context = self.context_emb(context)
        x = torch.cat((self.node_emb(x.squeeze(-1)), self.pe_lin(x_pe), time_emb.squeeze(1), context), dim=1)
        #print("Shape of node attr after concatenation:", x.shape)

        edge_attr = self.edge_emb(edge_attr.squeeze(-1))
        #print("Edge_attr shape before time:", edge_attr.shape)
        required_repeats = edge_attr.size(0) // time_emb.size(0)
        remainder = edge_attr.size(0) % time_emb.size(0)
        time_emb_edge = time_emb.repeat_interleave(required_repeats, dim=0)

        if remainder > 0:
          extra_time_emb = time_emb[:remainder]
        time_emb_edge = torch.cat((time_emb_edge, extra_time_emb), dim=0)


        assert time_emb_edge.size(0) == edge_attr.size(0), "Final sizes still do not match!"

        #print("shape of edge_attr:", edge_attr.shape)
        #print("Shape of time_emb_edge:", time_emb_edge.shape)

        edge_attr = torch.cat((edge_attr, time_emb_edge.squeeze(1)), dim=-1)
        #print("Shape of edge attr after concatenation:", edge_attr.shape)

        for conv in self.convs:
            if self.model_type == 'gine':
                x = conv(x, edge_index, edge_attr=edge_attr)
            else:
                x = conv(x, edge_index, batch, edge_attr=edge_attr)

        x = global_add_pool(x, batch)

        return self.mlp(x)

In [85]:
print("Train Loader Batches:")
for batch_index, batch in enumerate(train_loader):
    print(f"Batch {batch_index + 1}")
    #num_graphs = batch.ptr.size(0) - 1
    #print(num_graphs)
    #print(batch)

Train Loader Batches:
Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10
Batch 11
Batch 12
Batch 13
Batch 14
Batch 15
Batch 16
Batch 17
Batch 18
Batch 19
Batch 20
Batch 21
Batch 22
Batch 23
Batch 24
Batch 25
Batch 26
Batch 27
Batch 28
Batch 29
Batch 30
Batch 31
Batch 32
Batch 33
Batch 34
Batch 35
Batch 36
Batch 37
Batch 38
Batch 39
Batch 40
Batch 41
Batch 42
Batch 43
Batch 44
Batch 45
Batch 46
Batch 47
Batch 48
Batch 49
Batch 50
Batch 51
Batch 52
Batch 53
Batch 54
Batch 55
Batch 56
Batch 57
Batch 58
Batch 59
Batch 60
Batch 61
Batch 62
Batch 63
Batch 64
Batch 65
Batch 66
Batch 67
Batch 68
Batch 69
Batch 70
Batch 71
Batch 72
Batch 73
Batch 74
Batch 75
Batch 76
Batch 77
Batch 78
Batch 79
Batch 80
Batch 81
Batch 82
Batch 83
Batch 84
Batch 85
Batch 86
Batch 87
Batch 88
Batch 89
Batch 90
Batch 91
Batch 92
Batch 93
Batch 94
Batch 95
Batch 96
Batch 97
Batch 98
Batch 99
Batch 100
Batch 101
Batch 102
Batch 103
Batch 104
Batch 105
Batch 106
Batch 107
Batch 108
Batch 

In [86]:
# Model parameters
channels = 64
pe_dim = 20
num_layers = 12
model_type = 'mamba'  # Options: 'gine', 'mamba', 'transformer'
shuffle_ind = 10
d_state = 32
d_conv = 32
order_by_degree = False
time_embed_dim = 16
context_dim = 16

model = GraphModel(
    channels=channels,
    pe_dim=pe_dim,
    num_layers=num_layers,
    model_type=model_type,
    shuffle_ind=shuffle_ind,
    d_state=d_state,
    d_conv=d_conv,
    order_by_degree=order_by_degree,
    time_embed_dim=time_embed_dim,
    context_dim=context_dim
)

print(model)

GraphModel(
  (node_emb): Embedding(28, 12)
  (context_emb): Linear(in_features=1, out_features=16, bias=True)
  (pe_lin): Linear(in_features=20, out_features=20, bias=True)
  (pe_norm): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (edge_emb): Embedding(4, 64)
  (convs): ModuleList(
    (0-11): 12 x GPSConv(64, conv=GINEConv(nn=Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
    )), heads=4)
  )
  (mlp): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=1, bias=True)
  )
)


In [87]:
batch = next(iter(train_loader))
#print(batch.keys())
#print(batch_size)
x= (batch['x'])
pe = (batch['pe'])
edge_index = (batch['edge_index'])
edge_attr = (batch['edge_attr'])
batch_ = (batch['batch'])
num_graphs = (batch['ptr'].shape[0]-1)
print(num_graphs)

context = torch.randn_like(x, dtype=torch.float)
#print("Shape of context before:", context.shape)
context = context.view(-1, context.size(-1))
#print(f"Context shape after repeating: {context.shape}")

t = torch.randint(0, 1000, (num_graphs,)).float()
#t = t.view(-1, 1)
#t = t[batch_]
#print("t is:",t.shape)

output = model(x, pe, edge_index, t, context, edge_attr, batch_)

32
Shape of node attr after concatenation: torch.Size([770, 64])


ValueError: Node and edge feature dimensionalities do not match. Consider setting the 'edge_dim' attribute of 'GINEConv'

In [ ]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.pe, data.edge_index, data.edge_attr,
                    data.batch)
        loss = (out.squeeze() - data.y).abs().mean()
        loss.backward()
        total_loss += loss.item() * data.num_graphs
        optimizer.step()
    return total_loss / len(train_loader.dataset)

In [ ]:
@torch.no_grad()
def test(loader):
    model.eval()

    total_error = 0
    for data in loader:
        data = data.to(device)
        # print(data.x.shape)
        out = model(data.x, data.pe, data.edge_index, data.edge_attr,
                    data.batch)
        total_error += (out.squeeze() - data.y).abs().sum().item()
    return total_error / len(loader.dataset)

In [ ]:
# it.to(device)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GraphModel(channels=64, pe_dim=8, num_layers=10,
                   model_type='mamba',
                   shuffle_ind=0, order_by_degree=True,
                   d_conv=4, d_state=16,
                  ).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20,
                              min_lr=0.00001)
arr = []
for epoch in range(1, 30):
    loss = train()
    val_mae = test(val_loader)
    test_mae = test(test_loader)
    scheduler.step(val_mae)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val: {val_mae:.4f}, '
          f'Test: {test_mae:.4f}')
    arr.append(test_mae)
ordering = arr
print(ordering)

torch.Size([735, 64])
torch.Size([735, 64])
torch.Size([735, 64])
torch.Size([735, 64])
torch.Size([735, 64])
torch.Size([735, 64])
torch.Size([735, 64])
torch.Size([735, 64])
torch.Size([735, 64])
torch.Size([735, 64])
torch.Size([32, 64])
torch.Size([799, 64])
torch.Size([799, 64])
torch.Size([799, 64])
torch.Size([799, 64])
torch.Size([799, 64])
torch.Size([799, 64])
torch.Size([799, 64])
torch.Size([799, 64])
torch.Size([799, 64])
torch.Size([799, 64])
torch.Size([32, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([32, 64])
torch.Size([717, 64])
torch.Size([717, 64])
torch.Size([717, 64])
torch.Size([717, 64])
torch.Size([717, 64])
torch.Size([717, 64])
torch.Size([717, 64])
torch.Size([717, 64])
torch.Size([717, 64])
torch.Size([717, 64])
torch.Size([32, 64])
torch.Size([729, 64])
torch.Size([72

KeyboardInterrupt: 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GraphModel(channels=64, pe_dim=8, num_layers=10,
                   model_type='mamba',
                   shuffle_ind=1, order_by_degree=False,
                   d_conv=4, d_state=16,
                  ).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20,
                              min_lr=0.00001)
arr = []
for epoch in range(1, 30):
    loss = train()
    val_mae = test(val_loader)
    test_mae = test(test_loader)
    scheduler.step(val_mae)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val: {val_mae:.4f}, '
          f'Test: {test_mae:.4f}')
    arr.append(test_mae)
permute = arr
print(permute)

torch.Size([753, 64])
torch.Size([753, 64])
torch.Size([753, 64])
torch.Size([753, 64])
torch.Size([753, 64])
torch.Size([753, 64])
torch.Size([753, 64])
torch.Size([753, 64])
torch.Size([753, 64])
torch.Size([753, 64])
torch.Size([32, 64])
torch.Size([728, 64])
torch.Size([728, 64])
torch.Size([728, 64])
torch.Size([728, 64])
torch.Size([728, 64])
torch.Size([728, 64])
torch.Size([728, 64])
torch.Size([728, 64])
torch.Size([728, 64])
torch.Size([728, 64])
torch.Size([32, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([732, 64])
torch.Size([32, 64])
torch.Size([745, 64])
torch.Size([745, 64])
torch.Size([745, 64])
torch.Size([745, 64])
torch.Size([745, 64])
torch.Size([745, 64])
torch.Size([745, 64])
torch.Size([745, 64])
torch.Size([745, 64])
torch.Size([745, 64])
torch.Size([32, 64])
torch.Size([734, 64])
torch.Size([73

KeyboardInterrupt: 

In [ ]:
# import matplotlib.pyplot as plt
# import pandas as pd

# import numpy as np
# res_df = pd.read_csv('30_ep_res.csv')

# WINDOW = 1
# fig, ax = plt.subplots(1, figsize=(15,5))

# for col in res_df.columns:
#     plt.plot(res_df[col].clip(0,0.7).rolling(WINDOW, min_periods=1).mean(), label=col)

# plt.legend()
# plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, figsize=(15,5))

plt.plot(permute[:20], label='permute')
plt.plot(ordering[:20], label='order')

plt.legend()
plt.show()

In [ ]:
fffffffffff

# Plotting

# Tests

In [ ]:
ggggggggggg

In [ ]:
it = next(iter(train_loader))
# h, mask = to_dense_batch(it.x, it.batch)
# it.x.shape, h.shape, mask.shape

In [ ]:
deg.dtype

In [ ]:
it

In [ ]:
it.edge_index[0]

In [ ]:
it.to(device)
out = model(it.x, it.pe, it.edge_index, it.edge_attr,
                    it.batch)

In [ ]:
batch = torch.tensor([0,0,0,1,1,1,1])
x = torch.tensor([0,1,2,3,4,5,6])
batch.shape, x.shape

In [ ]:
import torch

def permute_within_batch(x, batch):
    # Enumerate over unique batch indices
    unique_batches = torch.unique(batch)

    # Initialize list to store permuted indices
    permuted_indices = []

    for batch_index in unique_batches:
        # Extract indices for the current batch
        indices_in_batch = (batch == batch_index).nonzero().squeeze()

        # Permute indices within the current batch
        permuted_indices_in_batch = indices_in_batch[torch.randperm(len(indices_in_batch))]

        # Append permuted indices to the list
        permuted_indices.append(permuted_indices_in_batch)

    # Concatenate permuted indices into a single tensor
    permuted_indices = torch.cat(permuted_indices)

    return permuted_indices

# Example usage
batch = torch.tensor([0, 0, 0, 1, 1, 1, 1])
x = torch.tensor([0, 10, 20, 30, 40, 50, 60])

# Get permuted indices
permuted_indices = permute_within_batch(x, batch)

# Use permuted indices to get the permuted tensor
permuted_x = x[permuted_indices]

print("Original x:", x)
print("Permuted x:", permuted_x)
print("Permuted indices:", permuted_indices)

Original x: tensor([ 0, 10, 20, 30, 40, 50, 60])
Permuted x: tensor([ 0, 10, 20, 60, 50, 30, 40])
Permuted indices: tensor([0, 1, 2, 6, 5, 3, 4])


In [ ]:
mask[0].sum(), (it.batch==0).sum()

In [ ]:
self_attn = Mamba(d_model=64, # Model dimension d_model
                                d_state=16,  # SSM state expansion factor
                                d_conv=4,    # Local convolution width
                                expand=1,    # Block expansion factor
                            )
print(sum(p.numel() for p in self_attn.parameters() if p.requires_grad), sum(p.numel() for p in self_attn.parameters()))

In [ ]:
self_attn = Mamba(d_model=64, # Model dimension d_model
                                d_state=8,  # SSM state expansion factor
                                d_conv=2,    # Local convolution width
                                expand=1,    # Block expansion factor
                            )
print(sum(p.numel() for p in self_attn.parameters() if p.requires_grad), sum(p.numel() for p in self_attn.parameters()))

In [ ]:
self_attn = Mamba(d_model=64, # Model dimension d_model
                                d_state=16,  # SSM state expansion factor
                                d_conv=8,    # Local convolution width
                                expand=1,    # Block expansion factor
                            )
print(sum(p.numel() for p in self_attn.parameters() if p.requires_grad), sum(p.numel() for p in self_attn.parameters()))

In [ ]:
self_attn = torch.nn.MultiheadAttention(
                64,
                4,
                dropout=0.5,
                batch_first=True,
            )
print(sum(p.numel() for p in self_attn.parameters() if p.requires_grad), sum(p.numel() for p in self_attn.parameters()))